In [20]:
import json

filename = 'data.json'

def read_data(filename):


    with open(filename, "r") as file:
        data = json.load(file)

    names = list(dict.fromkeys([row['data'] for row in data]))
    unique_queries = set([row['sql'] for row in data])

    dataset = {name:[] for name in names}
    for name in names:
        dataset[name] = [(row['question'], row['sql']) for row in data if row['data'] == name]

    return dataset, unique_queries

datasets, unique_queries = read_data(filename)
print(datasets)
print(unique_queries)

{'train': [('What are all the courses ?', 'SELECT name FROM course ;'), ('What are all the course codes ?', 'SELECT code FROM course ;')], 'dev': [('What are all the courses ?', 'SELECT name FROM course ;')], 'test': [('Please give me the names of courses .', 'SELECT name FROM course ;')]}
{'SELECT code FROM course ;', 'SELECT name FROM course ;'}


In [21]:
(
    {
    'train': [
        ('What are all the courses ?', 'SELECT name FROM course ;'),
        ('What are all the course codes ?', 'SELECT code FROM course ;')],
    'dev': [
        ('What are all the courses ?', 'SELECT name FROM course ;')],
    'test': [
        ('Please give me the names of courses .', 'SELECT name FROM course ;')]
    },
    {'SELECT name FROM course ;', 'SELECT code FROM course ;'}
)

({'train': [('What are all the courses ?', 'SELECT name FROM course ;'),
   ('What are all the course codes ?', 'SELECT code FROM course ;')],
  'dev': [('What are all the courses ?', 'SELECT name FROM course ;')],
  'test': [('Please give me the names of courses .',
    'SELECT name FROM course ;')]},
 {'SELECT code FROM course ;', 'SELECT name FROM course ;'})

In [22]:
dataset = {
    'train': [
        ('What are all the courses ?', 'SELECT name FROM course ;'),
        ('What are all the course codes ?', 'SELECT code FROM course ;')
    ],
    'dev': [
        ('What are all the courses ?', 'SELECT name FROM course ;')
    ],
    'test': [
        ('Please give me the names of courses .', 'SELECT name FROM course ;'),
        ('Please give the locations of courses .', 'SELECT location FROM course ;')
    ]
}

labels = {
    'SELECT name FROM course ;',
    'SELECT code FROM course ;',
    'SELECT location FROM course ;'
}

In [23]:
from collections import Counter
# This is the class you need to implement
class CodeModel:
    def __init__(self, labels, training_data):
        """Prepare the class member variables.
        Save the labels in self.labels and initialise all the weights to 0.

        Keyword arguments:
        labels -- a set of strings, each string is one SQL queryasd
        training_data -- a list, each item is a tuple containing a question and an SQL query
        """
        # TODO
        self.labels = list(labels)
        self.vocabulary = []
        self.weights = []
        for i in training_data:
            self.vocabulary += i[0].split()
        self.vocabulary = list(set(self.vocabulary))
        for i in list(labels):
            self.weights.append([0 for _ in self.vocabulary])
    
    def get_features(self, question, label):
        """Produce a list of features for a specific question and label.
        
        Keyword arguments:
        question -- a string, an English question
        label -- a string, an SQL query
        """
        # TODO
        features = [(i, label) for i in question.split()]
        return features

    def get_score(self, question, label):
        """Calculate the model's score for a question, label pair.
        
        Keyword arguments:
        question -- a string, an English question
        label -- a string, an SQL query
        """
        # TODO
        bow = [0 for _ in self.vocabulary]
        counter = Counter(question.split())
        for key, value in counter.items():
            if key in self.vocabulary:
                bow[self.vocabulary.index(key)] = value
        column = self.weights[self.labels.index(label)]
        return sum(a * b for a, b in zip(bow, column))

    def update(self, question, label, change):
        """Modify the model.
        Changes all weights for features for the (question, SQL query) pair by the amount indicated.

        Keyword arguments:
        question -- a string, an English question
        label -- a string, an SQL query
        change -- an integer, how much to change the weights
        """
        # TODO
        features = self.get_features(question, label)
        for i, j in features:  
            if i in self.vocabulary:
                self.weights[self.labels.index(j)][self.vocabulary.index(i)] += change
        return

q = 'Please list courses'
sql = 'SELECT name FROM course ;'
model = CodeModel(labels, dataset['train'])
model.get_features(q, sql)
model.get_score(q, sql)
model.update(q, sql, 1)
model.update('the the the', sql, 1)
model.weights

[[0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 3, 0], [0, 0, 0, 0, 0, 0, 0, 0]]

In [25]:
def find_best_code(question, model):
    """Predicts the SQL for a question by using a model to try all possible labels.

    Keyword arguments:
    question -- a string, the English question
    model -- a CodeModel, as defined in the Model question
    """
    # TODO
    scores = []
    for label in model.labels:
        score = model.get_score(question, label)
        scores.append(score)
    max_value = max(scores)
    idx = [list(model.labels)[i] for i, num in enumerate(scores) if num == max_value]
    idx.sort()
    return idx[0]

find_best_code(q, model)

'SELECT name FROM course ;'

In [26]:
def learn(question, answer, model, find_best_code):
    """Updates a model by predicting the SQL for a question and making a Perceptron update 

    Keyword arguments:
    question -- a string, the English question
    answer -- a string, the correct SQL query for this question 
    model -- a CodeModel, as defined in the Model question
    find_best_code -- a function, the one defined the Inference question
    """
    # TODO
    pred = find_best_code(question, model)
    if pred != answer:
        model.update(question, answer, 1)
        model.update(question, pred, -1)

q = 'What are all the course codes ?'
sql = 'SELECT code FROM course ;'
learn(q, sql, model, find_best_code)
model.weights

[[0, 1, 1, 1, 1, 1, 1, 1],
 [1, -1, -1, -1, -1, -1, 2, -1],
 [0, 0, 0, 0, 0, 0, 0, 0]]

In [27]:

eval_data = [
    ("What are all the courses ?", "SELECT name FROM course ;"),
    ("What are all the course codes ?", "SELECT code FROM course ;"),
    ("What are all the course names ?", "SELECT name FROM course ;"),
    ("Please give me the names of courses .", "SELECT name FROM course ;")
]


In [28]:
# This is the function you need to implement
def get_confusion_matrix(eval_data, model, find_best_code):
    """Creates a confusion matrix by predicting the SQL for a question and recording how the answer compares with the true answer 

    Keyword arguments:
    eval_data -- a list of tuples containing the English question and the true SQL query
    model -- a CodeModel, as defined in the Model question
    find_best_code -- a function, the one defined the Inference question
    """
    # TODO
    matrix = {}
    tuples = []
    for question, label in eval_data:
        pred = find_best_code(question, model)
        tuples.append((label, pred))
    for i in model.labels:
        for j in model.labels:
            matrix[(i, j)] = tuples.count((i, j))
    return matrix

matrix = get_confusion_matrix(eval_data, model, find_best_code)
matrix

{('SELECT code FROM course ;', 'SELECT code FROM course ;'): 1,
 ('SELECT code FROM course ;', 'SELECT name FROM course ;'): 0,
 ('SELECT code FROM course ;', 'SELECT location FROM course ;'): 0,
 ('SELECT name FROM course ;', 'SELECT code FROM course ;'): 2,
 ('SELECT name FROM course ;', 'SELECT name FROM course ;'): 1,
 ('SELECT name FROM course ;', 'SELECT location FROM course ;'): 0,
 ('SELECT location FROM course ;', 'SELECT code FROM course ;'): 0,
 ('SELECT location FROM course ;', 'SELECT name FROM course ;'): 0,
 ('SELECT location FROM course ;', 'SELECT location FROM course ;'): 0}

In [33]:
# These are the functions you need to implement
def calculate_accuracy(confusion_matrix, labels):
    """Returns the accuracy based on the contents of a confusion matrix

    Keyword arguments:
    confusion_matrix -- a dictionary, as defined in the Confusion Matrix question
    labels -- a set of strings, all the possible labels
    """
    # TODO
    count = sum(value for keys, value in confusion_matrix.items() if keys[0] == keys[1])
    total = sum(i for i in confusion_matrix.values())
    accuracy = count/total if total != 0 else 0 
    return accuracy

def calculate_precision(confusion_matrix, labels):
    """Returns a dict containing the precision for each label based on the contents of a confusion matrix

    Keyword arguments:
    confusion_matrix -- a dictionary, as defined in the Confusion Matrix question
    labels -- a set of strings, all the possible labels
    """
    # TODO
    precisions = {}
    for label in labels:
        column = [(key, value) for key, value in confusion_matrix.items() if key[1] == label]
        count = sum(value for keys, value in column if keys[0] == keys[1])
        total = sum(value for keys, value in column)
        precision = 1.0 if (count == 0 and total == 0) else count/total
        precisions[label] = precision
    return precisions 

def calculate_recall(confusion_matrix, labels):
    """Returns a dict containing the recall for each label based on the contents of a confusion matrix

    Keyword arguments:
    confusion_matrix -- a dictionary, as defined in the Confusion Matrix question
    labels -- a set of strings, all the possible labels
    """
    # TODO
    recalls = {}
    for label in labels:
        column = [(key, value) for key, value in confusion_matrix.items() if key[0] == label]
        count = sum(value for keys, value in column if keys[0] == keys[1])
        total = sum(value for keys, value in column)
        recall = 1.0 if (count == 0 and total == 0) else count/total
        recalls[label] = recall
    return recalls

def calculate_macro_f1(confusion_matrix, labels):
    """Returns the Macro F-Score based on the contents of a confusion matrix

    Keyword arguments:
    confusion_matrix -- a dictionary, as defined in the Confusion Matrix question
    labels -- a set of strings, all the possible labels
    """
    # TODO
    f1s = []
    precisions = calculate_precision(confusion_matrix, labels)
    recalls = calculate_recall(confusion_matrix, labels)
    for p, r in zip(precisions.values(), recalls.values()):
        f1 = 1.0 if (p+r == 0 and p*r == 0) else 2*p*r/(p+r)
        f1s.append(f1)
    macro_f1 = sum(f1s)/len(f1s)
    return macro_f1

calculate_accuracy(matrix, model.labels)
calculate_precision(matrix, model.labels)
calculate_recall(matrix, model.labels)
calculate_macro_f1(matrix, model.labels)

0.6666666666666666

In [34]:
# This is the function you need to implement
def main(filename, iterations, read_data, model_maker, learn, find_best_code, get_confusion_matrix, calculate_accuracy, calculate_macro_f1):
    """Trains and evaluates a model on some read_data

    Keyword arguments:
    filename -- a string, the location of a json file containing data
    iterations -- an integer, the number of iterations of training to do
    read_data -- a function, as defined in the Data question
    model_maker -- a class, as defined in the Model question
    learn -- a function, as defined in the Learning question
    find_best_code -- a function, as defined in the Inference question
    get_confusion_matrix -- a function, as defined in the Confusion Matrix question
    calculate_accuracy -- a function, as defined in the Evaluation Metrics question
    calculate_macro_f1 -- a function, as defined in the Evaluation Metrics question
    """
    # TODO
    dev_scores = []
    datasets, labels = read_data(filename)
    print(len(datasets['train']), len(datasets['dev']), len(datasets['test']))
    model = model_maker(labels, datasets['train'])
    for i in range(iterations):
        print(i)
        j = 0
        for question, sql in datasets['train']:
            learn(question, sql, model, find_best_code)
            #print(question)
            #if j > 100:
            #    break
            j += 1
        eval_matrix = get_confusion_matrix(datasets['dev'], model, find_best_code)
        eval_accuracy = calculate_accuracy(eval_matrix, labels)
        eval_macro_f1 = calculate_macro_f1(eval_matrix, labels)
        dev_scores.append({'accuracy': eval_accuracy, 'macro-f1': eval_macro_f1})
        print({'accuracy': eval_accuracy, 'macro-f1': eval_macro_f1})
    
    test_matrix = get_confusion_matrix(datasets['test'], model, find_best_code)
    test_accuracy = calculate_accuracy(test_matrix, labels)
    test_macro_f1 = calculate_macro_f1(test_matrix, labels)

    return dev_scores, {'accuracy': test_accuracy, 'macro-f1': test_macro_f1}

main('data_full.json', 10, read_data, CodeModel, learn, find_best_code, get_confusion_matrix, calculate_accuracy, calculate_macro_f1)

2629 229 573
0
{'accuracy': 0.27510917030567683, 'macro-f1': 0.5180516651104886}
1
{'accuracy': 0.5676855895196506, 'macro-f1': 0.6745144017202841}
2
{'accuracy': 0.7074235807860262, 'macro-f1': 0.7672424525365702}
3
{'accuracy': 0.7248908296943232, 'macro-f1': 0.7722660409192917}
4
{'accuracy': 0.8165938864628821, 'macro-f1': 0.8200414509238039}
5
{'accuracy': 0.8253275109170306, 'macro-f1': 0.8519883711060181}
6
{'accuracy': 0.8908296943231441, 'macro-f1': 0.8935942053589113}
7
{'accuracy': 0.9039301310043668, 'macro-f1': 0.909607277254336}
8
{'accuracy': 0.925764192139738, 'macro-f1': 0.9248443821973233}
9
{'accuracy': 0.9344978165938864, 'macro-f1': 0.9378073451602863}


([{'accuracy': 0.27510917030567683, 'macro-f1': 0.5180516651104886},
  {'accuracy': 0.5676855895196506, 'macro-f1': 0.6745144017202841},
  {'accuracy': 0.7074235807860262, 'macro-f1': 0.7672424525365702},
  {'accuracy': 0.7248908296943232, 'macro-f1': 0.7722660409192917},
  {'accuracy': 0.8165938864628821, 'macro-f1': 0.8200414509238039},
  {'accuracy': 0.8253275109170306, 'macro-f1': 0.8519883711060181},
  {'accuracy': 0.8908296943231441, 'macro-f1': 0.8935942053589113},
  {'accuracy': 0.9039301310043668, 'macro-f1': 0.909607277254336},
  {'accuracy': 0.925764192139738, 'macro-f1': 0.9248443821973233},
  {'accuracy': 0.9344978165938864, 'macro-f1': 0.9378073451602863}],
 {'accuracy': 0.8726003490401396, 'macro-f1': 0.8473756091403151})

In [ ]:
([{'accuracy': 0.29694323144104806, 'macro-f1': 0.18153134826504286},
  {'accuracy': 0.6200873362445415, 'macro-f1': 0.364145059733295},
  {'accuracy': 0.6593886462882096, 'macro-f1': 0.3875778089013383},
  {'accuracy': 0.7860262008733624, 'macro-f1': 0.4653241638535756},
  {'accuracy': 0.8165938864628821, 'macro-f1': 0.48174178762414055},
  {'accuracy': 0.8296943231441049, 'macro-f1': 0.4951532128002716},
  {'accuracy': 0.8820960698689956, 'macro-f1': 0.5275676937441643},
  {'accuracy': 0.8908296943231441, 'macro-f1': 0.536221811957106},
  {'accuracy': 0.8951965065502183, 'macro-f1': 0.5371594092182328},
  {'accuracy': 0.8820960698689956, 'macro-f1': 0.5305845570551453}],
 {'accuracy': 0.8830715532286213, 'macro-f1': 0.7946755858520564})